In [3]:
library(dplyr)
library(ggplot2)
library(stringr)
library(viridis)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: viridisLite



In [4]:
jz.trim.masked.primer <- function(data) {
  # Vectorize the operations on the SEQUENCE_INPUT column
  data$SEQUENCE_TRIM = str_sub(data$SEQUENCE_INPUT, data$V_SEQ_START, str_length(data$SEQUENCE_INPUT))
  data$SEQUENCE_TRIM = str_sub(data$SEQUENCE_TRIM, 1, data$J_SEQ_START - data$V_SEQ_START + data$J_SEQ_LENGTH)

  # Count the number of 'N' characters in each row
  data$N_count = str_count(data$SEQUENCE_TRIM, 'N')

  # Calculate the fraction of 'N' characters in each row
  data$N_frac = data$N_count / str_length(data$SEQUENCE_TRIM)

  return(data)
}

In [5]:
# April 29 2024 second novaseq run 240410Lov
bcr = read.table('vdj_contigs/00_combined_vdj_contig.tab',sep='\t',header=TRUE)

In [6]:
bcr = jz.trim.masked.primer(bcr)
summary(bcr$N_count)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   0.00    0.00    0.00    0.19    0.00   63.00    4234 

In [7]:
fil.bcr = bcr %>% filter(FUNCTIONAL == TRUE, R1CONSCOUNT>=5, R2CONSCOUNT>=5,N_count<6)

In [8]:
summary(fil.bcr$R1CONSCOUNT)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      5      17      39     361      91  125084 

In [9]:
fil.bcr$barcode = str_sub(fil.bcr$SEQUENCE_ID, 1, 16)
fil.bcr$sample = str_sub(fil.bcr$SAMPLE, 1, nchar(fil.bcr$SAMPLE)-1)

fil.bcr$cell.id = paste(fil.bcr$sample,fil.bcr$barcode,sep='_')
fil.bcr$chain = ifelse(fil.bcr$ISOTYPE %in% c('IGKC','IGLC_1','IGLC_2') & fil.bcr$LOCUS %in% c('IGK','IGL'), 'light',
                       ifelse((! fil.bcr$ISOTYPE %in% c('IGKC','IGLC_1','IGLC_2')) & fil.bcr$LOCUS =='IGH','heavy','mixed') )
fil.bcr = fil.bcr %>% filter(chain != 'mixed')
head(fil.bcr)

,SEQUENCE_ID,SEQUENCE_INPUT,FUNCTIONAL,IN_FRAME,STOP,MUTATED_INVARIANT,INDELS,LOCUS,V_CALL,D_CALL,⋯,PVAL,SAMPLE,ID,SEQUENCE_TRIM,N_count,N_frac,barcode,sample,cell.id,chain
,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,CACAACATCCCACAGGCCCTTAATCGAG,NNNNNNNNNNNNNNNNNNNNNNNNNCGCTGTCTCTGGTTACTCCATCAGCAGTGGTTACTACTGGGGCTGGATCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATCATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGTCAACATGTAACGGGGAGCTATCCGAAAGTTGTTTCGACCCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGNNNNNNNNNN,TRUE,TRUE,FALSE,NA,FALSE,IGH,IGHV4-38-2*01,"IGHD3-10*01,IGHD3-10*02,IGHD3-16*02",⋯,1.1102e-16,H1H,H1H_CACAACATCCCACAGGCCCTTAATCGAG,CGCTGTCTCTGGTTACTCCATCAGCAGTGGTTACTACTGGGGCTGGATCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATCATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCCGTGTATTACTGTGCGTCAACATGTAACGGGGAGCTATCCGAAAGTTGTTTCGACCCCTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,0,0,CACAACATCCCACAGG,H1,H1_CACAACATCCCACAGG,heavy
2,GGGTTATGTTCTCGCTGATGTAGCGACA,NNNNNNNNNNNNNNNNNNNNNNNNNGCTTCTGGATACACCTTCACCGGCTACTATATGCACTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGATGGATCAACCCTAACAGTGGTGGCACAAACTATGCACAGAAGTTTCAGGGCAGGGTCACCATGACCAGGGACACGTCCATCAGCACAGCCTACATGGAGCTGAGCAGGCTGAGATCTGACGACACGGCCGTGTATTACTGTGCGAGAGTTACCACTATATTACGATTTTTGGAGTGGTCCTTGGGGTACTGGGGCCAGGGAACGCTGGTCACCGTCTCCTCAGNNNNNNNNNN,TRUE,TRUE,FALSE,NA,FALSE,IGH,IGHV1-2*02,IGHD3-3*01,⋯,1.1102e-16,H1H,H1H_GGGTTATGTTCTCGCTGATGTAGCGACA,GCTTCTGGATACACCTTCACCGGCTACTATATGCACTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGATGGATCAACCCTAACAGTGGTGGCACAAACTATGCACAGAAGTTTCAGGGCAGGGTCACCATGACCAGGGACACGTCCATCAGCACAGCCTACATGGAGCTGAGCAGGCTGAGATCTGACGACACGGCCGTGTATTACTGTGCGAGAGTTACCACTATATTACGATTTTTGGAGTGGTCCTTGGGGTACTGGGGCCAGGGAACGCTGGTCACCGTCTCCTCAG,0,0,GGGTTATGTTCTCGCT,H1,H1_GGGTTATGTTCTCGCT,heavy
3,AGGATAAGTTGGCTATAGAACCGCTCTA,NNNNNNNNNNNNNNNNNNNNNNNNNTGTAGCCTCTGAATTCAGCTTTACTCAGTATTGGATGGCCTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTGGCCAATATAAGACAAGATGGAAATATTCAGATTTATGCTGACTCTGTGAGGGGCCGATTCACCATCTCCAGAGACAACGTCAACAGTTTTCTGTATCTGCACATGGACGCCCTAAGAGTCGATGACACGGCTATGTATTATTGTGCGAGAGTAGACGGTTCCTTCCATGGCGCGGACTACTGGGGCCAGGGAACCCAGGTCACCGTCTCGTCAGNNNNNNNNN,TRUE,TRUE,FALSE,NA,FALSE,IGH,IGHV3-7*01,"IGHD3-10*01,IGHD3-10*03,IGHD4-17*01",⋯,1.1102e-16,H1H,H1H_AGGATAAGTTGGCTATAGAACCGCTCTA,TGTAGCCTCTGAATTCAGCTTTACTCAGTATTGGATGGCCTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTGGCCAATATAAGACAAGATGGAAATATTCAGATTTATGCTGACTCTGTGAGGGGCCGATTCACCATCTCCAGAGACAACGTCAACAGTTTTCTGTATCTGCACATGGACGCCCTAAGAGTCGATGACACGGCTATGTATTATTGTGCGAGAGTAGACGGTTCCTTCCATGGCGCGGACTACTGGGGCCAGGGAACCCAGGTCACCGTCTCGTCAG,0,0,AGGATAAGTTGGCTAT,H1,H1_AGGATAAGTTGGCTAT,heavy
4,AGTAGTCAGTCATCCACCTCGTATTATT,NNNNNNNNNNNNNNNNNNNNNNNNNTGCAGCGTCTGGATTCACCTTCAGTAGCTATGGCATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCATTTATACGGTATGATGGAAGTAATAAATACTACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAGCCTGAGAGCTGAGGACACGGCTGTGTATTACTGTGCCCCATCCCCGGGTTACTATGATAGTAGTGGTTACTTCTTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGNNNNNNNNNN,TRUE,TRUE,FALSE,NA,FALSE,IGH,"IGHV3-30*02,IGHV3-30-5*02",IGHD3-22*01,⋯,1.1102e-16,H1H,H1H_AGTAGTCAGTCATCCACCTCGTATTATT,TGCAGCGTCTGGATTCACCTTCAGTAGCTATGGCATGCACTGGGTCCGCCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCATTTATACGGTATGATGGAAGTAATAAATACTACGCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAGCCTGAGAGCTGAGGACACGGCTGTGTATTACTGTGCCCCATCCCCGGGTTACTATGATAGTAGTGGTTACTTCTTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,0,0,AGTAGTCAGTCATCCA,H1,H1_AGTAGTCAGTCATCCA,heavy
5,GTCACTCTCATGCCGGGCCCATTGGGTT,NNNNNNNNNNNNNNNNNNNNNNNNNGCTTCTGGTTACACCTTTACCAACTATGATATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGATGGATCAGCGCTTCCACTGGTTACACAAACGATGCACAGAAGTTCCAGGGCAGAGTCACCATGACCGCAGACACATCCACGAGCACAGCCTACATGGAGCTGAGGAGCCTGAGATCTGACGACACGGCCATGTATTACTGTGCGAGAGCCCGAGGGGGCAGTGGCTGGTACGGTCACTACTTTGACTACTGGGGCCAGGGAACGCTGGTCACCGTCTC

In [10]:
table(fil.bcr$LOCUS, fil.bcr$ISOTYPE)

     
        IGHA   IGHD IGHG1/2   IGHM   IGKC IGLC_1 IGLC_2
  IGH   7312  84316    9436  81730      0      0      0
  IGK      0      0       0      0 300034      0      0
  IGL      0      0       0      0      0 111560 261434

In [11]:
write.csv(fil.bcr,'vdj_contigs/01_filtered_vdj_contig.csv',row.names = FALSE)